In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.06G/1.06G [00:05<00:00, 226MB/s]
100% 1.06G/1.06G [00:05<00:00, 204MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Loading pretrained VGG16 Model

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
conv_base.trainable = True


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Finetuning only specific layers

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
    if set_trainable:
      layer.trainable = True
    layer.trainable = set_trainable

In [ ]:
# Building the model

model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [ ]:
# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Generator

train_ds = train_datagen.flow_from_directory(
    directory='/content/train',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

validation_ds = val_datagen.flow_from_directory(
    directory='/content/test',
    target_size=(256,256),
    batch_size=32,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
# Compile the model
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Early stopping

early_stopping = EarlyStopping(monitor = 'val_loss', patience=3)

In [ ]:
# Train the Model

history = model.fit(train_ds, epochs=20, validation_data=validation_ds, callbacks=[early_stopping])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 578s 838ms/step - accuracy: 0.8717 - loss: 0.2764 - val_accuracy: 0.9490 - val_loss: 0.1433
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 531s 771ms/step - accuracy: 0.9696 - loss: 0.0756 - val_accuracy: 0.9812 - val_loss: 0.0455
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 486s 745ms/step - accuracy: 0.9799 - loss: 0.0538 - val_accuracy: 0.9880 - val_loss: 0.0320
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 470s 747ms/step - accuracy: 0.9826 - loss: 0.0452 - val_accuracy: 0.9858 - val_loss: 0.0380
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 502s 746ms/step - accuracy: 0.9841 - loss: 0.0388 - val_accuracy: 0.9840 - val_loss: 0.0450
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 480s 763ms/step - accuracy: 0.9876 - loss: 0.0336 - val_accuracy: 0.9806 - val_loss: 0.0605


In [ ]:
# Saving the model

model.save("cats_and_dogs_vgg_model.h5")
print("Model saved as cats_and_dogs_vgg_model.h5")

Model saved as cats_and_dogs_vgg_model.h5


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


In [ ]:
# Load the trained model
model = load_model("cats_and_dogs_vgg_model.h5")

In [ ]:
# Define class labels
class_names = ["Cat", "Dog"]

In [ ]:
# Streamlit app
st.title("Cats and Dogs Classification")
st.write("Upload an image to classify it as a Cat or Dog.")

2025-01-13 13:20:59.674 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:20:59.799 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-01-13 13:20:59.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:20:59.804 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:20:59.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:20:59.808 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:20:59.810 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

2025-01-13 13:21:03.102 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:21:03.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:21:03.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:21:03.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-13 13:21:03.114 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
if uploaded_file is not None:
    # Display the image
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    st.write("Classifying...")

In [ ]:
if uploaded_file is not None:
    # Display the image
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    st.write("Classifying...")
    # Preprocess the image
    image = load_img(uploaded_file, target_size=(256, 256))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    # Make prediction
    prediction = model.predict(image_array)
    predicted_class = class_names[int(prediction[0] > 0.5)]

    # Display the result
    st.write(f"The image is classified as: **{predicted_class}**")

In [ ]:
app_code = """
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model = load_model("cats_and_dogs_vgg_model.h5")

# Define class labels
class_names = ["Cat", "Dog"]

# Streamlit app
st.title("Cats and Dogs Classification")
st.write("Upload an image to classify it as a Cat or Dog.")

# Upload image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the image
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    st.write("Classifying...")

    # Preprocess the image
    image = load_img(uploaded_file, target_size=(256, 256))
    image_array = img_to_array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    # Make prediction
    prediction = model.predict(image_array)
    predicted_class = class_names[int(prediction[0] > 0.5)]

    # Display the result
    st.write(f"The image is classified as: **{predicted_class}**")
"""
# Save to app.py
with open("app.py", "w") as f:
    f.write(app_code)
print("app.py file created!")


app.py file created!


In [ ]:
from google.colab import files
files.download("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mv app.py /content/drive/My\ Drive/
print("app.py saved to Google Drive.")

Mounted at /content/drive
app.py saved to Google Drive.
